# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import pandas as pd
import logging
import json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        responses = []
        preds = scoring_func(inputString)
        responses.append(preds)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Writing driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [4]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [5]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

'check length of multidimensional arrays with javascript.  possible duplicate: length of javascript associative array   i want to check the length of a multidimensional array but i get "undefined" as the return. i\'m assuming that i am doing something wrong with my code but i can\'t see anything odd about it.  thoughts? could this have something to do with scope? the array is declared and set outside of the function. could this have something to do with json? i created the array from an eval() statement. why does the dummy array work just fine?'

Here, we define a helper function to convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [6]:
def text_to_json(text):
    return json.dumps({'input':'{0}'.format(text)})

In [7]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [8]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 272.56 ms


In [9]:
resp = predict_for(body)

INFO:model_driver:Predictions: [[(14220321, 14220323, 0.9606876834677185), (11922383, 11922384, 0.4081062445074803), (23667086, 23667087, 0.31746740503812876), (5223, 6700, 0.05185960075914204), (6847697, 6847754, 0.021318976490582196), (1885557, 1885660, 0.021055101665837), (750486, 750506, 0.0191493082245365), (7837456, 14853974, 0.018846690399866205), (1789945, 1789952, 0.016315270828631684), (1451009, 1451043, 0.011258634514542357), (7486085, 7486130, 0.009691468803469574), (1584370, 1584377, 0.006466570431909596), (19590865, 19590901, 0.006359408978479496), (1068834, 1144249, 0.005596260596406045), (1634268, 1634321, 0.005546208984470831), (5767325, 5767357, 0.005340908050389464), (901115, 901144, 0.004880565449463552), (20279484, 20279485, 0.004815453647972756), (18082, 1830844, 0.004778375349940621), (3434278, 3434388, 0.004729790074929237), (6259982, 6260001, 0.0042945368948937345), (950087, 950146, 0.004175894023129493), (10693845, 10693852, 0.003781132521957991), (122102, 122

Next, we move on to building our docker image.